In [1]:
import sys
!{sys.executable} -m pip install PyAthena


You are using pip version 10.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
from pyathena import connect
import pandas as pd
conn = connect(s3_staging_dir='s3://aws-athena-query-results-984073016564-us-west-2/sagemaker', region_name='us-west-2')
df = pd.read_sql("SELECT distinct * FROM retsdata.union_geo3 WHERE county = 'San Diego'", conn)

In [3]:
len(df)

473450

In [4]:
df[:10]

,streetnumber,streetdirprefix,streetname,city,state,zip5,directions,cmplx,neighborhd,county
0,7745,None,Northern Lights,San Diego,CA,92127,Cross Street: Encendido.,Santaluz,Santaluz,San Diego
1,14859,None,Fox Hunt Lane,San Diego,CA,92128,Cross Street: Heather Glen.,Carmel Mountain Ranch,Carmel Mountain Ranch,San Diego
2,11342,None,Turtleback Ln,San Diego,CA,92127,Bernardo Center- Cloudcrest Dr- L to Turtlebac...,High Country West,Rancho Bernardo,San Diego
3,2544,None,Navarra Dr.,Carlsbad,CA,92009,Take La Costa Ave. East * Left on Viejo Castil...,La Costa Fairways,Carlsbad South,San Diego
4,7923,None,Avenida Diestro,Carlsbad,CA,92009,RSF Rd/East on Camino de la Coches/R on Terazz...,La Costa,La Costa,San Diego
5,5038,None,Via Papel,San Diego,CA,92122,"Governor Drive, south on Lakewood, left on Via...",University City,University City,San Diego
6,4822,None,Mount Almagosa Dr,San Diego,CA,92111,"From the 5 freeway head East on Balboa Ave, Ri...",Clairemont,Clairemont,San Diego
7,1645,None,Haydn Dr and Summit Ave,Cardiff by the Sea,CA,92007,Cross Street: Liszt Ave.,Composer District,Cardiff By The Sea,San Diego
8,2078,None,James Gaynor St,Fallbrook,CA,92028,Cross Street: Tom McGuiness Jr. Cir.,Fallbrook,Fallbrook,San Diego
9,614,None,Dorothy St,El Cajon,CA,92019,Cross Street: Washington.,Granite Hills,El Cajon,San Diego


In [5]:
import tensorflow as tf
import numpy as np

In [6]:
def pad(df):
    return (df['streetnumber'].map(lambda s: '{!s:6.6}'.format(str(s))) + ' ' + 
       df['streetname'].map(lambda s: '{!s:20.20}'.format(s).upper()) + ' ' + 
       df['city'].map(lambda s: '{!s:16.16}'.format(s).upper()))

df_train   = pad(df.loc[df['zip5'] != '0'])
df_labels  = df['zip5'].map(str)
df_predict = pad(df.loc[df['zip5'] == '0'])

In [7]:
df_train[:10]

0    7745   NORTHERN LIGHTS      SAN DIEGO       
1    14859  FOX HUNT LANE        SAN DIEGO       
2    11342  TURTLEBACK LN        SAN DIEGO       
3    2544   NAVARRA DR.          CARLSBAD        
4    7923   AVENIDA DIESTRO      CARLSBAD        
5    5038   VIA PAPEL            SAN DIEGO       
6    4822   MOUNT ALMAGOSA DR    SAN DIEGO       
7    1645   HAYDN DR AND SUMMIT  CARDIFF BY THE S
8    2078   JAMES GAYNOR ST      FALLBROOK       
9    614    DOROTHY ST           EL CAJON        
dtype: object

In [8]:
df_labels[:10]

0    92127
1    92128
2    92127
3    92009
4    92009
5    92122
6    92111
7    92007
8    92028
9    92019
Name: zip5, dtype: object

In [9]:
df_predict[:10]

Series([], dtype: object)

In [10]:
df_train[:3]

0    7745   NORTHERN LIGHTS      SAN DIEGO       
1    14859  FOX HUNT LANE        SAN DIEGO       
2    11342  TURTLEBACK LN        SAN DIEGO       
dtype: object

In [11]:
N_CODES = 128

def encode(df):
    sess = tf.Session()
    v = df.map(lambda s: list(map(ord, list(s)))).values.tolist()
    v = sess.run(list(map(lambda row: tf.one_hot(row, N_CODES), v)))
    v = list(map(lambda v: [flatten for sub in v for flatten in sub], v))
    return pd.Series(v)
    

In [12]:
encode(df_train[:3])

0    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
1    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
2    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
dtype: object

In [13]:
SUBSET=1000
train_inputs = encode(df_train[:SUBSET])

In [14]:
train_inputs[:3]

0    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
1    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
2    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
dtype: object

In [15]:
train_labels = encode(df_labels[:SUBSET])
train_labels[:3]

0    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
1    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
2    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
dtype: object

In [16]:
len(train_labels[0])

640

In [17]:
SAMPLES = len(train_inputs)
INPUT_VAR_CODES = len(train_inputs[0])
INPUT_VARS = INPUT_VAR_CODES // N_CODES
OUTPUT_VAR_CODES = len(train_labels[0])
OUTPUT_VARS = OUTPUT_VAR_CODES // N_CODES

In [18]:
X = tf.placeholder(tf.float32, shape=[1, len(train_inputs[0])])
Y = tf.placeholder(tf.float32, shape=[1, len(train_labels[0])])

In [19]:
HIDDEN=5
parameters = {
		'W1': tf.Variable(tf.random_normal([len(train_inputs[0]), HIDDEN])),
		'b1': tf.Variable(tf.random_normal([HIDDEN])),
		'W2': tf.Variable(tf.random_normal([HIDDEN, len(train_labels[0])])),
		'b2': tf.Variable(tf.random_normal([len(train_labels[0])]))
}

In [20]:
num_epochs = 2000
learning_rate = 0.0005

In [21]:
def neural_net(X,parameters):
	Z1 = tf.add(tf.matmul(X, parameters['W1']), parameters['b1'])
	A2 = tf.nn.relu(Z1)
	Z2 = tf.add(tf.matmul(A2, parameters['W2']), parameters['b2'])
	return Z2

In [24]:
import datetime

def train():
    print (datetime.datetime.now())
    Z = neural_net(X,parameters)
    costs = []
    optimizers = []
    for i in range(OUTPUT_VARS):
        c = Z[0][N_CODES*i:N_CODES*i+N_CODES]
        costs.append(tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=c,  labels=Y[0][N_CODES*i:N_CODES*i+N_CODES])))
        optimizer_k = optimizers.append(tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(costs[i]))
    optimizer = tf.group(*optimizers)
    cost = tf.reduce_sum(costs)
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

    with tf.Session() as sess:
        sess.run(init)
        epoch = 0
        while epoch<num_epochs:
            for i in range(SAMPLES):
                _ , c = sess.run([optimizer, cost], feed_dict={
                    X: np.reshape(train_inputs[i],[1,INPUT_VAR_CODES]), 
                    Y: np.reshape(train_labels[i],[1,OUTPUT_VAR_CODES])
                }) 
            print (str(datetime.datetime.now()) +  " Cost after epoch %i: %f" % (epoch, c))
            epoch += 1
        saver.save(sess, 'model.ckpt')


In [ ]:
train()

2019-09-30 06:35:37.607209
2019-09-30 06:35:43.720593 Cost after epoch 0: 21.939651
2019-09-30 06:35:47.280504 Cost after epoch 1: 18.651726
2019-09-30 06:35:50.849037 Cost after epoch 2: 15.585085
2019-09-30 06:35:54.393590 Cost after epoch 3: 12.867194
2019-09-30 06:35:57.908330 Cost after epoch 4: 10.600194
2019-09-30 06:36:01.469536 Cost after epoch 5: 8.841795
2019-09-30 06:36:05.229356 Cost after epoch 6: 7.582443
2019-09-30 06:36:08.816782 Cost after epoch 7: 6.734311
2019-09-30 06:36:12.353835 Cost after epoch 8: 6.181186
2019-09-30 06:36:15.969085 Cost after epoch 9: 5.826673
2019-09-30 06:36:19.539582 Cost after epoch 10: 5.601499
2019-09-30 06:36:23.104584 Cost after epoch 11: 5.458176
2019-09-30 06:36:26.684180 Cost after epoch 12: 5.365606
2019-09-30 06:36:30.266886 Cost after epoch 13: 5.304727
2019-09-30 06:36:33.920965 Cost after epoch 14: 5.264065
2019-09-30 06:36:37.523027 Cost after epoch 15: 5.236790
2019-09-30 06:36:41.261310 Cost after epoch 16: 5.218990
2019-09-3

In [ ]:
def toheno(onehot):
    # reverse the one-hot encoding
    calcs = []
    for i in range(len(onehot)//N_CODES):
        charvec = onehot[N_CODES*i : N_CODES*(i+1)]
        calcs.append(tf.argmax(charvec))
    with tf.Session() as sess:
        out = sess.run(calcs)
    return ''.join([chr(i) for i in out])
        
toheno(train_inputs[0])        

In [ ]:
def test():
    # Test predictions by computing the output using training set as input
    with tf.Session() as sess:
        init = tf.global_variables_initializer()
        sess.run(init)
        saver = tf.train.import_meta_graph('model.ckpt.meta')
        saver.restore(sess,'model.ckpt')
        for row in range(len(train_inputs)):

            g = train_inputs[row]
            g = np.reshape(g,[1,INPUT_VAR_CODES])
            output = neural_net(g,parameters)
            outputs = []
            for i in range(OUTPUT_VARS):
                kk = tf.nn.softmax(output[0][N_CODES*i : N_CODES*i+N_CODES])
                outputs.append(kk)
                
            
            out = sess.run(outputs)
            out = np.reshape(list(map(list,out)), [1, OUTPUT_VAR_CODES])[0].tolist()
            out = list(map(lambda x: float("%.1f" % x), out))
            
            print("\nROW #" + str(row))
            print("Expected: " + toheno(train_labels[row]))
            print("Actual..: " + toheno(out))            
            row = row + 1


In [ ]:
test()